In [3]:
import gym
import numpy as np

%matplotlib notebook
from matplotlib import pyplot

from tqdm import tqdm_notebook as tqdm

In [ ]:
class nn():
    def __init__(l_config, alpha, activation_func):
        self.a = [np.zeros((i,1)) for i in l_config]
        self.w = [np.random.rand(l_config[i-1],l_config[i]) for i in range(1,len(l_config))]
        self.alpha = alpha
        self.activation = self.select_actiivation(activation_func)
        
    def select_actiivation(self, key):
        if key=='RELU':
            return self.relu
        elif key=='SIGMOID':
            return self.sigmoid
    
    def sofmax(self, a):
        return np.exp(a-np.max(a))/np.sum(np.exp(a-np.max(a)))
    
    def forward(self, x):
        a[0] = np.reshape(x, self.a[0].shape)
        
        for i in range(len(self.w)):
            z = self.w[i].T @ self.a[i]
            a[i+1] = np.reshape(self.activation(z), self.a[i+1].shape)
            
        return sofmax(a[-1])  

In [9]:
type(np.random.rand(1,2).shape)

tuple

In [14]:
f = lambda a: a==1; True

True